<b>Setup -- read in dataset, create SQLite database, populate tables</b>

In [8]:
import xlrd
import pandas as pd
import sqlite3
import statistics

# Read Excel wrksheets into dataframes
opportunity_list = pd.read_excel('data/dataset.xlsx', sheet_name='Opportunity List', header=0)
net_transaction_revenue = pd.read_excel('data/dataset.xlsx', sheet_name='Net Transaction Revenue', header=0)

# Create SQLite database
sql_connection = sqlite3.connect('data/fs_database.db')
cursor = sql_connection.cursor()

# Note - SQLite doesn't support date, bool, decimal types
cursor.execute( 
    """
    CREATE TABLE IF NOT EXISTS opportunity_list (
        business_id INTEGER,
        event_type TEXT NOT NULL,
        event_date TEXT NOT NULL,
        pay_merchant_flag INTEGER NOT NULL,
        amount INTEGER NOT NULL,
        PRIMARY KEY(business_id)
        );
    """
)

cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS net_transaction_revenue (
        business_id INTEGER,
        calendar_month TEXT NOT NULL,
        net_transaction_revenue INTEGER NOT NULL,
        FOREIGN KEY(business_id) REFERENCES opportunity_list(business_id)
        );
    """
)

cursor.close()

# Populate SQL tables with database from Excel
opportunity_list.to_sql('opportunity_list', sql_connection, if_exists='replace', index=False)
net_transaction_revenue.to_sql('net_transaction_revenue', sql_connection, if_exists='replace', index=False)

<hr />

<b>#1 - How many new deals were sold each month?</b>

In [2]:
pd.read_sql(
    """
    SELECT strftime('%Y-%m', date(event_date)) as Month, count(*) as NumberSold
    FROM opportunity_list
    WHERE event_type LIKE 'New Sale'
    GROUP BY strftime('%Y-%m', date(event_date))
    """
    , sql_connection)

,Month,NumberSold
0,2020-01,321
1,2020-02,394
2,2020-03,184
3,2020-04,183
4,2020-05,163
5,2020-06,205
6,2020-07,340
7,2020-08,306
8,2020-09,241
9,2020-10,297


<b>#2 - What % of total deals are considered Pay deals in May 2020?</b>

In [3]:
# Remainder of integer division is discarded, so SUM / COUNT without the casts would return 0
pd.read_sql(
    """
    SELECT ROUND(CAST(SUM(pay_merchant_flag) AS REAL)/CAST(COUNT(*) AS REAL) * 100,2) || '%' as PercentPayDeals
    FROM opportunity_list
    WHERE strftime('%Y-%m', date(event_date)) LIKE '2020-05'
    """
    , sql_connection)

,PercentPayDeals
0,11.98%


<b>#3 - What is the Month 3 total revenue generated and how has it changed from the initial month?</b>

In [4]:
merchants = pd.read_sql(
    """
    SELECT business_id, event_date, amount as InitialAmount, strftime('%Y-%m-%d', date(event_date)) AS MonthOne, strftime('%Y-%m-01', date(event_date, '+3 months')) AS MonthThree
    FROM opportunity_list
    WHERE event_type LIKE 'New Sale'
    """
    , sql_connection)

def GetRevenueChanges(biz_ID, EndDate):
    query = """
    SELECT business_id, SUM(net_transaction_revenue) as Rev 
    FROM net_transaction_revenue
    WHERE business_id=(?) AND (date(calendar_month) LIKE (?)) 
    GROUP BY business_id
    """
    queryResult = pd.read_sql(query, sql_connection, params=(biz_ID,EndDate))
    
    if len(queryResult) == 0:
        return 0
    else:
        return queryResult['Rev'][0]

result = []

for index, row in merchants.iterrows():
    change = GetRevenueChanges(row['business_id'], row['MonthThree'])
    result.append([row['business_id'], row['InitialAmount'], change, row['InitialAmount']+change])

dfResult = pd.DataFrame(result, columns = ['business_id', 'initial_amount', 'change', 'MonthThreeRevenue'])

totalNum = len(dfResult)
numChanged = len(dfResult[dfResult['change'] > 0])
avgChange = round(sum(dfResult['change']) / numChanged,2)
pctChanged = round((numChanged / len(dfResult)) * 100,0)
initialRev = sum(dfResult['initial_amount'])
totalRev = round(sum(dfResult['MonthThreeRevenue']),2)
diff = round(totalRev - initialRev,2)
print()
print(f"Month 3 total revenue generated is ${totalRev:,}, a change of +${diff:,} from month 1")
print(f"{pctChanged}% of merchants MRR changed from initial value in the first 3 months ({numChanged} / {totalNum}), for an average increase of +${avgChange}")
print()


Month 3 total revenue generated is $608,684.94, a change of +$38,601.0 from month 1
8.0% of merchants MRR changed from initial value in the first 3 months (263 / 3134), for an average increase of +$146.77



<b>#4 If stakeholders ask you what merchant type (pay vs loyalty) generates the most value to FS. In this case, value is defined as most revenue generated. What is your recommendation and why?</b>

In [5]:
payMerchants = pd.read_sql(
    """
    SELECT 
        printf("$%.2f", AVG(monthly.AverageMRR)) as AverageMRR,
        MAX(monthly.TotalCustomers) as TotalCustomers,
        printf("$%,d", AVG(monthly.TotalMRR)) as TotalMRR
    FROM
    (
        SELECT
            strftime('%Y-%m', date(n.calendar_month)) as Month,
            AVG(o.amount)+AVG(n.net_transaction_revenue) as AverageMRR,
            COUNT(DISTINCT o.business_id) as TotalCustomers,
            SUM(o.amount)+SUM(n.net_transaction_revenue) as TotalMRR
        FROM opportunity_list o
        LEFT JOIN net_transaction_revenue as n ON o.business_id=n.business_id
        WHERE o.event_type LIKE 'New Sale' AND o.pay_merchant_flag=1
        GROUP BY n.calendar_month
    ) as monthly;
    """
    , sql_connection)

print("Payments: ")
print(payMerchants)

loyaltyMerchants = pd.read_sql(
    """
    SELECT
        printf("$%.2f", AVG(o.amount)) as AverageMRR,
        COUNT(DISTINCT o.business_id) as TotalCustomers,
        printf("$%,d", SUM(o.amount)) as TotalMRR
    FROM opportunity_list o
    WHERE o.event_type LIKE 'New Sale' AND o.pay_merchant_flag=0
    """
    , sql_connection)

print("Loyalty: ")
print(loyaltyMerchants)
print()
print("On average a payments customer generates a 19% higher MRR than a loyalty customer, however in our current customer base 92% of revenue is generated by loyalty customers.")

Payments: 
  AverageMRR  TotalCustomers TotalMRR
0    $229.61             376  $46,998
Loyalty: 
  AverageMRR  TotalCustomers  TotalMRR
0    $192.16            2747  $528,240

On average a payments customer generates a 19% higher MRR than a loyalty customer, however in our current customer base 92% of revenue is generated by loyalty customers.
